# VEGAS in spinfoams: application on a toy model

In [15]:
using JupyterFormatter
enable_autoformat()

1-element Vector{Function}:
 format_current_cell (generic function with 1 method)

This toy model has the purpose of testing the VEGAS algorithm in order to compute a discret sum.

I consider a function $F(x_0, y_0, z_0)$ of 3 variables defined as:

\begin{align}
F(x_0, y_0, z_0) & = \sum\limits_{x = 0}^{10} \sum\limits_{y = 0}^{10} \sum\limits_{z = 0}^{10} e^{-\frac{1}{2} \left( x - x_0 \right)^2} e^{-\frac{1}{2} \left( y - y_0 \right)^2} e^{-\frac{1}{2} \left( z - z_0 \right)^2} \sin \left( xy \right) \cos \left( z \right) \\
& \equiv \sum\limits_{x = 0}^{10} \sum\limits_{y = 0}^{10} \sum\limits_{z = 0}^{10} f(x_0, y_0, z_0, x, y, z)
\end{align}

In [85]:
function F(x0, y0, z0)

    result = 0.0

    for x = 0:10, y = 0:10, z = 0:10
        result +=
            exp(-(1 / 2) * (x - x0)^2) *
            exp(-(1 / 2) * (y - y0)^2) *
            exp(-(1 / 2) * (z - z0)^2) *
            sin(x * y) *
            cos(z)
    end

    result

end

F (generic function with 1 method)

## Exact result

Let's first compute the exact value of the function for a chosen value of $(x_0, y_0, z_0)$:

In [86]:
@time F(2, 6, 10)

  0.000039 seconds


0.692056321795933

## Standard Monte Carlo

Let's now construct a standard Monte Carlo estimator for the above function

$$
\lim_{N\to\infty} F_{MC}(x_0, y_0, z_0, N) = F(x_0, y_0, z_0) 
$$

which is given by:

\begin{align}
F_{MC}(x_0, y_0, z_0, N) & = \frac{V}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} e^{-\frac{1}{2} \left( x - x_0 \right)^2} e^{-\frac{1}{2} \left( y - y_0 \right)^2} e^{-\frac{1}{2} \left( z - z_0 \right)^2} \sin \left( xy \right) \cos \left( z \right) \\
& \equiv \frac{V}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f(x_0, y_0, z_0, [x, y, z])
\end{align}

where $ V = 11^3 $, as this is the total number of possible configurations. 

With respect to the discrete case, in the continuous $V$ coincides with the volume of the subset of $\mathbb{R^3}$ on which the integration is performed.

In [17]:
using Distributions, Random

In [18]:
function FMC(x0, y0, z0, N, V)

    draw = Array{Int}(undef, 3)
    draw_float_sample = Array{Float64}(undef, 1)
    Uniform_distribution = Uniform(0, 10)

    result = 0.0

    for i = 1:N

        for i = 1:3
            rand!(Uniform_distribution, draw_float_sample)
            draw_float_sample[1] = round(Int64, draw_float_sample[1])
            draw[i] = Int(draw_float_sample[1])
        end

        result +=
            exp(-(1 / 2) * (draw[1] - x0)^2) *
            exp(-(1 / 2) * (draw[2] - y0)^2) *
            exp(-(1 / 2) * (draw[3] - z0)^2) *
            sin(draw[1] * draw[2]) *
            cos(draw[3])

    end

    result *= V / N

    result

end

FMC (generic function with 1 method)

In [19]:
V = 11^3;

In [68]:
@time FMC(2, 6, 10, 10^7, V)

  0.724211 seconds (4 allocations: 176 bytes)


0.6327959611211814

$\textbf{The converge is slow}$: we need $10^7$ iterations to see a result stable up to the first significant digit. 

More importantly, $\textbf{the converges is biased}$, since the sum is highly oscillating and has several cancellations due to the negative terms.

### Standard deviation

The standard deviations $\sigma^2_{F}$ $\sigma^2_{F_{MC}}$ (of $F$ and $F_{MC}$ respectively) are an indication of the possible error in the Monte Carlo estimate. 

Let's start by defining 

$$ 
\langle F \rangle \equiv \frac{1}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f(x_0, y_0, z_0, [x, y, z]) = \frac{1}{V}F_{MC}(x_0, y_0, z_0, N)
$$

Variance $\sigma^2_{F}$ can be estimated from the Monte Carlo data as:

\begin{align}
\sigma^2_{F} & \equiv \frac{1}{N-1} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} \left( f(x_0, y_0, z_0, [x, y, z]) - \langle F \rangle \right)^2 \\
 & =  \frac{1}{N-1} \left[ \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f(x_0, y_0, z_0, [x, y, z])^2 - N \langle F \rangle^2 \right] \\ 
\end{align}


From which $\sigma^2_{F_{MC}}$ is given by:

$$
\sigma^2_{MC} = \frac{V^2}{N} \sigma^2_{F} = \frac{V^2}{N-1} \left[ \frac{1}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f(x_0, y_0, z_0, [x, y, z])^2 - \langle F \rangle^2 \right]
$$

# Changing variables

I can perform a trivial change of variables in order to sample from the interval $[0,1]$:

$$
X = \frac{x}{10}, \ Y = \frac{y}{10},\  Z = \frac{z}{10}
$$

The function changes as:

$$
F(x_0, y_0, z_0) = \sum\limits_{X \in [0,1]} \sum\limits_{Y \in [0,1]} \sum\limits_{Z \in [0,1]} f(x_0, y_0, z_0, x(X), y(Y), z(Z))
$$

Since the original quantity was a sum and not a continuous integral, of course it is necessary to round.

The MC estimator becomes:

\begin{align}
F_{MC}(x_0, y_0, z_0, N) & = \frac{V}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} f(x_0, y_0, z_0, [x(X), y(Y), z(Z)])
\end{align}

In [20]:
function FMC_change_variable(x0, y0, z0, N, V)

    draw = Array{Int}(undef, 3)
    draw_float_sample = Array{Float64}(undef, 1)
    Uniform_distribution = Uniform(0, 1)

    result = 0.0

    for i = 1:N

        for i = 1:3
            rand!(Uniform_distribution, draw_float_sample)
            draw_float_sample[1] = 10 * draw_float_sample[1]
            draw_float_sample[1] = round(Int64, draw_float_sample[1])
            draw[i] = Int(draw_float_sample[1])
        end

        result +=
            exp(-(1 / 2) * (draw[1] - x0)^2) *
            exp(-(1 / 2) * (draw[2] - y0)^2) *
            exp(-(1 / 2) * (draw[3] - z0)^2) *
            sin(draw[1] * draw[2]) *
            cos(draw[3])

    end

    result *= V / N

    result

end

FMC_change_variable (generic function with 1 method)

In [76]:
@time FMC_change_variable(2, 6, 10, 10^7, V)

  0.765712 seconds (80.87 k allocations: 4.597 MiB, 3.64% compilation time)


0.6512009430339419

## Introducing a Jacobian

We can rewrite the above expression by introducing the following Jacobian for the coordinate transformation:

$$
J (X,Y,Z) = 
\begin{vmatrix}
\frac{d x}{d X} & 0 & 0 \\
0 & \frac{d y}{d Y} & 0 \\
0 & 0 & \frac{d z}{d Z} \\
\end{vmatrix}
=
10^3
$$

which leads to:

\begin{align}
F_{MC}(x_0, y_0, z_0, N) = \frac{V_J}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)])
\label{MC_estimator} \tag{1}
\end{align}

In the continuous case we have $V_J = 1$.

$$ 
\langle F \rangle_J \equiv \frac{1}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} J \cdot f(x_0, y_0, z_0, [x, y, z]) = \frac{1}{V_J}F_{MC}(x_0, y_0, z_0, N)
$$

Variance $\sigma^2_{F_J}$ is estimated as:

\begin{align}
\sigma^2_{F_J} & \equiv \frac{1}{N-1} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} \left( J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)]) - \langle F \rangle_J \right)^2 \\
 & =  \frac{1}{N-1} \left[ \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} J^2 \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)])^2 - N \langle F \rangle_J^2 \right] \\ 
\end{align}


Variance $\sigma^2_{F_{MC_J}}$ is estimated as:

$$
\sigma^2_{MC_J} = \frac{V_J^2}{N} \sigma^2_{F_J} = \frac{V_J^2}{N-1} \left[ \frac{1}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} J^2 \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)])^2 - \langle F \rangle_J^2 \right]
$$

## VEGAS

I write explicitly the equations considering a single axis for simplicity. 

For each axis, we have a grid with $a = x_0$, $b = x_{N_g}$, where $N_g$ is the total number of elements in the grid. 

$ x_0 = a $

$ x_1 = x_0 + \Delta x_0 $

$ x_2 = x_1 + \Delta x_1 $

$ \dots $

$ x_{N_g} = x_{N_g-1} + \Delta x_{N_g-1} = b $

Let's define a matrix which contains the grid over the 3 axes:

\begin{align}
\text{VEGAS_grid}^{T} = 
\begin{pmatrix}
x_0 & x_1 & \dots & x_{N_g} \\
y_0 & y_1 & \dots & y_{N_g}  \\
z_0 & z_1 & \dots & z_{N_g}  \\
\end{pmatrix}
\end{align}

and matrix for the corresponding spacing:

\begin{align}
\text{VEGAS_spacing_grid}^{T} = 
\begin{pmatrix}
\Delta x_0 & \Delta x_1 & \dots & \Delta x_{N_g-1} \\
\Delta y_0 & \Delta y_1 & \dots & \Delta y_{N_g-1}  \\
\Delta z_0 & \Delta z_1 & \dots & \Delta z_{N_g-1}  \\
\end{pmatrix}
\end{align}

The first natural choice for the grid is a linear spaced one. We therefore consider initially all equal $\Delta x_i = x_{i+1} - x_{i} = \frac{b-a}{N_g}$ for $i = 0 \dots N_g -1$.  

In [27]:
a = 0;
b = 10;
N_g = 100;
N_ev = 10^7

VEGAS_grid = Array{Float64}(undef, N_g + 1, 3);
VEGAS_grid_improved = Array{Float64}(undef, N_g + 1, 3);

for i = 1:3
    VEGAS_grid[:, i] .= collect(LinRange(a, b, N_g + 1))
end

#Delta_x_i = (b - a) / (N_g);

VEGAS_spacing_grid = Array{Float64}(undef, N_g, 3);
VEGAS_spacing_grid_improved = Array{Float64}(undef, N_g, 3);

#for i = 1:3
#    VEGAS_spacing_grid[:, i] .= [Delta_x_i for i = 1:1:N_g]
#end

for i = 1:3
    for j = 1:N_g
        VEGAS_spacing_grid[j, i] = VEGAS_grid[j+1, i] - VEGAS_grid[j, i]
    end
end

In [28]:
VEGAS_grid

101×3 Matrix{Float64}:
  0.0   0.0   0.0
  0.1   0.1   0.1
  0.2   0.2   0.2
  0.3   0.3   0.3
  0.4   0.4   0.4
  0.5   0.5   0.5
  0.6   0.6   0.6
  0.7   0.7   0.7
  0.8   0.8   0.8
  0.9   0.9   0.9
  1.0   1.0   1.0
  1.1   1.1   1.1
  1.2   1.2   1.2
  ⋮          
  8.9   8.9   8.9
  9.0   9.0   9.0
  9.1   9.1   9.1
  9.2   9.2   9.2
  9.3   9.3   9.3
  9.4   9.4   9.4
  9.5   9.5   9.5
  9.6   9.6   9.6
  9.7   9.7   9.7
  9.8   9.8   9.8
  9.9   9.9   9.9
 10.0  10.0  10.0

In [29]:
VEGAS_spacing_grid

100×3 Matrix{Float64}:
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 ⋮         
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1

The following function provides the map between $X \in [0,1]$ and $x \in [a, b]$

In [22]:
# Returns the VEGAS grid element and the corresponding grid index
function x(X, N_g, VEGAS_grid, VEGAS_spacing_grid)

    i_X = Int(floor(X * N_g))

    delta_X = X * N_g - i_X

    if (i_X == N_g)
        VEGAS_grid[i_X+1, 1], i_X + 1
    else
        VEGAS_grid[i_X+1, 1] + VEGAS_spacing_grid[i_X+1, 1] * delta_X, i_X + 1
    end

end


#=
function VEGAS_rounding_map_OLD!(draw1, draw2, X, N_g, VEGAS_grid, VEGAS_spacing_grid) 
    draw1[1] = round(Int64, floor(X*N_g))
    #println(draw[1])
    draw2[1] = VEGAS_grid[draw1[1]+1]
    #println(draw[2])
    if (draw[1] != N_g)
        draw2[1] += VEGAS_spacing_grid[draw1[1]+1]*(X*N_g - draw1[1])
    end
end
=#

x (generic function with 1 method)

In [12]:
@time x(1, N_g, VEGAS_grid, VEGAS_spacing_grid)

  0.000004 seconds (1 allocation: 32 bytes)


(10.0, 11)

The Jacobian for this transformation is easily computed as:

$$
J(X) = N_g \Delta x_{i(X)}  ≡ J_{i(X)}
$$

along each axis. This is a step function, and the values are determined by the interval widths $ \Delta x_{i} $.

In [5]:
#=
function MC_sampling_OLD(x0, y0, z0, N_ev, N_g, VEGAS_grid, VEGAS_spacing_grid)
    
    draw = Array{Int}(undef, 3);
    VEGAS_draw_1 = Array{Int}(undef, 1);
    VEGAS_draw_2 = Array{Float64}(undef, 1);
    draw_float_sample = Array{Float64}(undef, 1);
    Uniform_distribution = Uniform(0, 1);
    
    result = 0.0
   
    for i=1:N_ev
        
          for i = 1:3
              # this is X \in [0,1]
              rand!(Uniform_distribution, draw_float_sample)
              #println(draw_float_sample[1])
            
              VEGAS_rounding_map_OLD!(VEGAS_draw_1, VEGAS_draw_2, draw_float_sample[1], N_g, VEGAS_grid, VEGAS_spacing_grid)

              draw[i] = round(Int, VEGAS_draw_2[1])
          end
    
        result += exp(-(1/2)*(draw[1] - x0)^2)*exp(-(1/2)*(draw[2] - y0)^2)*exp(-(1/2)*(draw[3] - z0)^2)*sin(draw[1]*draw[2])*cos(draw[3])
        
    end
    
    result = result*11^3/N_ev
        
    return result
    
end
=#

In [6]:
#@time MC_sampling_OLD(2, 6, 10, N_ev, N_g, VEGAS_grid, VEGAS_spacing_grid)

### Refining the grid

Given some samples $[X, Y, Z]_1 \dots [X, Y, Z]_N$ and the corresponding Monte Carlo estimate $\eqref{MC_estimator}$, now I want to estimate the following quantities:

$$
d^x_0 = \frac{1}{n^x_0} \sum_{x(X) \in \Delta x_0} \left( J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)]) \right)^2
$$

$$
d^x_1 = \frac{1}{n^x_1} \sum_{x(X) \in \Delta x_1} \left( J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)]) \right)^2
$$

$$
\dots
$$

$$
d^x_{N_g-1} = \frac{1}{n^x_{N_g-1}} \sum_{x(X) \in \Delta x_{N_g-1}}  \left( J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)]) \right)^2
$$

etc. $\textbf{along each axis}$, where $n^x_i \approx \frac{N_{ev}}{N_g}$ is the number of samples in interval $\Delta x_i$.

Let's define a matrix:

\begin{align}
\text{VEGAS_d}= 
\begin{pmatrix}
d_0^x & d_1^x & \dots & d_{N_g-1}^x \\
d_0^y & d_1^y & \dots & d_{N_g-1}^y  \\
d_0^z & d_1^z & \dots & d_{N_g-1}^z  \\
\end{pmatrix}
\end{align}

It is also useful to consider a matrix to take into account the multeplicity:

\begin{align}
\text{VEGAS_d_multeplicity}= 
\begin{pmatrix}
n_0^x & n_1^x & \dots & n_{N_g-1}^x \\
n_0^y & n_1^y & \dots & n_{N_g-1}^y  \\
n_0^z & n_1^z & \dots & n_{N_g-1}^z  \\
\end{pmatrix}
\end{align}

In [51]:
VEGAS_d = zeros(3, N_g);

Result_estimate = Array{Float64}(undef, 1)

1-element Vector{Float64}:
 0.0

In [124]:
function VEGAS_sampling(
    x0,
    y0,
    z0,
    N_ev,
    N_g,
    VEGAS_grid,
    VEGAS_spacing_grid,
    VEGAS_d,
    V,
    Result_estimate,
)

    draw = Array{Int}(undef, 3)
    draw_float_sample = Array{Float64}(undef, 1)
    Uniform_distribution = Uniform(0, 1)

    VEGAS_index_draw = Array{Int}(undef, 3)
    VEGAS_d_multeplicity = zeros(3, N_g)
    Jacobians = Array{Float64}(undef, 3)
    VEGAS_index_draw = Array{Int}(undef, 3)

    total_result = 0.0

    for n = 1:N_ev

        for i = 1:3
            # this is X in [0,1]
            rand!(Uniform_distribution, draw_float_sample)
            println("The following number X was generated along axis $(i) between 0 and 1:")
            println(draw_float_sample[1])

            i_X = Int(floor(draw_float_sample[1] * N_g))
            delta_X = draw_float_sample[1] * N_g - i_X
            VEGAS_index_draw[i] = i_X_index = i_X + 1

            # this is x(X) in [a,b]        
            if (i_X == N_g)
                draw_float_sample[1] = VEGAS_grid[VEGAS_index_draw[i], i]
            else
                draw_float_sample[1] =
                    VEGAS_grid[VEGAS_index_draw[i], i] +
                    VEGAS_spacing_grid[VEGAS_index_draw[i], i] * delta_X
            end

            println(
                "The following number x is the corresponding in the grid along axis $(i) between 0 and $(N_g):",
            )
            println(draw_float_sample[1])

            println(
                "It corresponds to index $(i_X) in the grid, which means index $(VEGAS_index_draw[i]) in the matrix",
            )

            Jacobians[i] = N_g * VEGAS_spacing_grid[VEGAS_index_draw[i], i]
            println("This is the Jacobian along axis $(i):")
            println(Jacobians[i])

            VEGAS_d_multeplicity[i, VEGAS_index_draw[i]] += 1
            #println("This is the multeplicity of VEGAS_d:")
            #println(VEGAS_d_multeplicity)

            # we need to round to integers 
            draw_float_sample[1] = round(Int64, draw_float_sample[1])
            draw[i] = round(Int64, draw_float_sample[1])
            println("The number was rounded to:")
            println(draw[i])
            println(" ")
        end

        println(" ")
        println(" ")
        println(" ")

        result =
            exp(-(1 / 2) * (draw[1] - x0)^2) *
            exp(-(1 / 2) * (draw[2] - y0)^2) *
            exp(-(1 / 2) * (draw[3] - z0)^2) *
            sin(draw[1] * draw[2]) *
            cos(draw[3]) *
            prod(Jacobians)

        total_result += result

        d_result = result^2

        for i = 1:3
            VEGAS_d[i, VEGAS_index_draw[i]] += d_result
        end

    end

    # final normalization
    VEGAS_d[:] .= VEGAS_d[:] ./ VEGAS_d_multeplicity[:]

    total_result *= (V / 10^3) / N_ev

    Result_estimate[1] = total_result

end


VEGAS_sampling (generic function with 2 methods)

In [125]:
@time VEGAS_sampling(
    2,
    6,
    10,
    10,
    10,
    VEGAS_grid,
    VEGAS_spacing_grid,
    VEGAS_d,
    V,
    Result_estimate,
)

The following number X was generated along axis 1 between 0 and 1:
0.5206433952912467
The following number x is the corresponding in the grid along axis 1 between 0 and 10:
0.5206433952912467
It corresponds to index 5 in the grid, which means index 6 in the matrix
This is the Jacobian along axis 1:
0.9999999999999998
The number was rounded to:
1
 
The following number X was generated along axis 2 between 0 and 1:
0.0922784383543972
The following number x is the corresponding in the grid along axis 2 between 0 and 10:
0.0922784383543972
It corresponds to index 0 in the grid, which means index 1 in the matrix
This is the Jacobian along axis 2:
1.0
The number was rounded to:
0
 
The following number X was generated along axis 3 between 0 and 1:
0.7649000177857423
The following number x is the corresponding in the grid along axis 3 between 0 and 10:
0.7649000177857425
It corresponds to index 7 in the grid, which means index 8 in the matrix
This is the Jacobian along axis 3:
0.9999999999999

LoadError: DimensionMismatch("array could not be broadcast to match destination")

In [55]:
Result_estimate

1-element Vector{Float64}:
 0.6575033139997568

Now we can implement the classic VEGAS algorithm. 

$\textbf{From now on, I stop describing explicitly all steps since it takes too much time}$

In [36]:
function VEGAS_d_smoothing(N_g, VEGAS_d)

    VEGAS_sum_all_d = vec(sum(VEGAS_d, dims = 1))

    for i = 1:3

        VEGAS_d[1, i] = (7 * VEGAS_d[1, i] + VEGAS_d[2, i]) / (8 * VEGAS_sum_all_d[i])

        for n = 2:(N_g-1)
            VEGAS_d[n, i] =
                (VEGAS_d[n-1, i] + 6 * VEGAS_d[n, i] + VEGAS_d[n+1, i]) /
                (8 * VEGAS_sum_all_d[i])
        end

        VEGAS_d[N_g, i] =
            (VEGAS_d[N_g-1, i] + 7 * VEGAS_d[N_g, i]) / (8 * VEGAS_sum_all_d[i])
    end

end

VEGAS_d_smoothing (generic function with 1 method)

In [37]:
function VEGAS_d_compression(N_g, VEGAS_d, alpha)

    for i = 1:3
        for n = 1:N_g
            VEGAS_d[n, i] = ((1 - VEGAS_d[n, i]) / (log(1 / VEGAS_d[n, i])))^(alpha)
        end
    end

end

VEGAS_d_compression (generic function with 1 method)

In [121]:
function VEGAS_grid_refinement(
    N_g,
    VEGAS_d,
    VEGAS_grid,
    VEGAS_grid_improved,
    VEGAS_spacing_grid,
    VEGAS_spacing_grid_improved,
)

    delta_d = Array{Float64}(undef, 3)

    VEGAS_sum_all_d = vec(sum(VEGAS_d, dims = 1))

    println(VEGAS_sum_all_d)

    delta_d[:] .= VEGAS_sum_all_d[:] ./ N_g

    for axis = 1:3

        VEGAS_grid_improved[1, axis] = VEGAS_grid[1, axis]
        VEGAS_grid_improved[end, axis] = VEGAS_grid[end, axis]

        # index of current improved x
        i = 0

        # index of current old x
        j = 0

        # amount of d accumulated
        S_d = 0

        while (true)

            i += 1

            # CHECK INDEX AT THE END
            if (i == N_g)
                break
            end

            while (S_d < delta_d[axis])

                S_d += VEGAS_d[j+1, axis]

                #  println(S_d)
                #  println(delta_d[axis])
                #  println(j)

                j += 1

            end

            S_d -= delta_d[axis]

            println(i)
            println(j)
            flush(stdout)

            VEGAS_grid_improved[i+1, axis] =
                VEGAS_grid[j+1, axis] -
                (S_d / VEGAS_d[j, axis]) * VEGAS_spacing_grid[j, axis]

        end

        for j = 1:N_g
            VEGAS_spacing_grid_improved[j, axis] =
                VEGAS_grid_improved[j+1, axis] - VEGAS_grid_improved[j, axis]
        end

    end

end

VEGAS_grid_refinement (generic function with 2 methods)

Notice that it is better to reshape the matrix as:

\begin{align}
\text{VEGAS_d} \longrightarrow
\begin{pmatrix}
d_0^x & d_0^y & d_0^z \\
d_1^x & d_1^y & d_1^z \\
\vdots & \vdots & \vdots \\
d_{N_g-1}^x & d_{N_g-1}^y & d_{N_g-1}^z \\
\end{pmatrix}
\end{align}

according to julia's convention for data storage

In [43]:
VEGAS_d = permutedims(VEGAS_d, (2, 1))

VEGAS_d_smoothing(N_g, VEGAS_d)

VEGAS_d_compression(N_g, VEGAS_d, 1.5)

VEGAS_grid_refinement(
    N_g,
    VEGAS_d,
    VEGAS_grid,
    VEGAS_grid_improved,
    VEGAS_spacing_grid,
    VEGAS_spacing_grid_improved,
)

VEGAS_d = permutedims(VEGAS_d, (2, 1))

3×100 Matrix{Float64}:
 0.0       0.0        0.0      0.0600551  …  0.0432306  0.0432325  0.0432304
 0.0       0.0        0.0      0.0371799     0.0630299  0.0630303  0.0630313
 0.042069  0.0413344  0.04131  0.0415965     0.188985   0.188465   0.188548

In [44]:
VEGAS_d

3×100 Matrix{Float64}:
 0.0       0.0        0.0      0.0600551  …  0.0432306  0.0432325  0.0432304
 0.0       0.0        0.0      0.0371799     0.0630299  0.0630303  0.0630313
 0.042069  0.0413344  0.04131  0.0415965     0.188985   0.188465   0.188548

In [45]:
VEGAS_grid_improved

101×3 Matrix{Float64}:
  0.0        0.0        0.0
  0.421976   0.493513   0.189559
  0.50136    0.652708   0.380445
  0.569712   0.811713   0.563703
  0.637846   0.970759   0.745425
  0.705799   1.12983    0.927136
  0.773681   1.28891    1.10884
  0.841653   1.44196    1.29056
  0.909669   1.58338    1.46773
  0.977603   1.72307    1.63886
  1.04555    1.8628     1.8097
  1.11348    2.00252    1.98055
  1.18138    2.14231    2.15139
  ⋮                    
  8.04502    8.62985    9.53854
  8.20816    8.74727    9.58069
  8.37131    8.86464    9.62272
  8.5387     8.98195    9.66464
  8.71666    9.0993     9.70655
  8.89492    9.2167     9.7484
  9.07317    9.33411    9.79025
  9.25141    9.45267    9.83219
  9.42993    9.58371    9.87415
  9.61624    9.72243    9.91611
  9.80811    9.86122    9.95805
 10.0       10.0       10.0

In [46]:
VEGAS_spacing_grid_improved

100×3 Matrix{Float64}:
 0.421976   0.493513  0.189559
 0.0793845  0.159195  0.190886
 0.0683516  0.159005  0.183258
 0.0681338  0.159046  0.181722
 0.0679537  0.159076  0.181711
 0.0678817  0.159079  0.181708
 0.067972   0.153042  0.181711
 0.0680156  0.14142   0.177172
 0.0679341  0.139692  0.171132
 0.067944   0.139732  0.17084
 0.0679379  0.139719  0.170849
 0.0678936  0.139788  0.170838
 0.0680748  0.13972   0.170584
 ⋮                    
 0.163151   0.116976  0.0425471
 0.163136   0.117421  0.0421505
 0.163148   0.117368  0.0420277
 0.167398   0.117308  0.0419238
 0.177957   0.117351  0.0419121
 0.17826    0.117397  0.0418487
 0.178247   0.117408  0.0418487
 0.178238   0.118561  0.0419374
 0.17853    0.131046  0.0419643
 0.186306   0.138717  0.0419572
 0.191874   0.138786  0.0419457
 0.191885   0.138784  0.0419457

## Full VEGAS algorithm

Let's try now to write the full algorithm 

Let's try now to write the full algorithm 

The list of parameters is the following:

- $x_0$, $y_0$ $z_0$: Input numbers of function $F$
- $N_g$: Number of elements in the grid
- $N_{ev}$: Number of Monte Carlo sample in the grid for each iteration
- $N_{it}$: Number of VEGAS iterations (at each iteration the grid improves)
- $\alpha$: Parameter used in the compression phase
- $V$: Number of total configurations (used to re-normalize in the discrete case)

In [122]:
function VEGAS_algorithm(x0, y0, z0, N_g, N_ev, N_it, alpha, V)


    VEGAS_grid = Array{Float64}(undef, N_g + 1, 3)
    VEGAS_grid_improved = Array{Float64}(undef, N_g + 1, 3)

    a = 0
    b = 10
    for i = 1:3
        VEGAS_grid[:, i] .= collect(LinRange(a, b, N_g + 1))
    end

    VEGAS_spacing_grid = Array{Float64}(undef, N_g, 3)
    VEGAS_spacing_grid_improved = Array{Float64}(undef, N_g, 3)

    for i = 1:3
        for j = 1:N_g
            VEGAS_spacing_grid[j, i] = VEGAS_grid[j+1, i] - VEGAS_grid[j, i]
        end
    end

    Result_estimate = Array{Float64}(undef, 1)

    for n = 1:N_it

        println("VEGAS iteration $(n)")
        flush(stdout)

        VEGAS_d = zeros(3, N_g)

        # populates VEGAS_d and Result_estimate by using current VEGAS_grid and VEGAS_spacing_grid
        VEGAS_sampling(
            x0,
            y0,
            z0,
            N_ev,
            N_g,
            VEGAS_grid,
            VEGAS_spacing_grid,
            VEGAS_d,
            V,
            Result_estimate,
        )

        println("The current estimate of the sum is $(Result_estimate[1])")
        flush(stdout)

        VEGAS_d = permutedims(VEGAS_d, (2, 1))

        VEGAS_d_smoothing(N_g, VEGAS_d)

        VEGAS_d_compression(N_g, VEGAS_d, alpha)

        # refines the grid by using VEGAS_d and the current VEGAS_grid and VEGAS_spacing_grid
        VEGAS_grid_refinement(
            N_g,
            VEGAS_d,
            VEGAS_grid,
            VEGAS_grid_improved,
            VEGAS_spacing_grid,
            VEGAS_spacing_grid_improved,
        )

        VEGAS_d = permutedims(VEGAS_d, (2, 1))

        #println(VEGAS_spacing_grid[:, 2])

        VEGAS_spacing_grid[:] .= VEGAS_spacing_grid_improved[:]

        VEGAS_grid[:] .= VEGAS_grid_improved[:]

    end

end

VEGAS_algorithm (generic function with 1 method)

In [123]:
VEGAS_algorithm(2, 6, 10, 10, 10, 10, 1, V)

VEGAS iteration 1
The following number X was generated along axis 1 between 0 and 1:
0.31805787452236
The following number x is the corresponding in the grid along axis 1 between 0 and 10:
3.1805787452236
It corresponds to index 3 in the grid, which means index 4 in the matrix
This is the Jacobian along axis 1:
10.0
The number was rounded to:
3
 
The following number X was generated along axis 2 between 0 and 1:
0.07498007254998962
The following number x is the corresponding in the grid along axis 2 between 0 and 10:
0.7498007254998962
It corresponds to index 0 in the grid, which means index 1 in the matrix
This is the Jacobian along axis 2:
10.0
The number was rounded to:
1
 
The following number X was generated along axis 3 between 0 and 1:
0.5501067191205411
The following number x is the corresponding in the grid along axis 3 between 0 and 10:
5.501067191205411
It corresponds to index 5 in the grid, which means index 6 in the matrix
This is the Jacobian along axis 3:
10.0
The number

LoadError: BoundsError: attempt to access 10×3 Matrix{Float64} at index [0, 1]